In [ ]:
# qiskit_runtime.ipynb

from pathlib import Path

from matplotlib import pyplot as plt
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import QiskitRuntimeService

# Cell 1 - Save IBM API Token

my_api_token = "you_api_key_here"

# Saves your IBM Quantum credential to $HOME/.qiskit/qiskit-ibm.json
QiskitRuntimeService.save_account(
    channel="ibm_quantum",
    token=my_api_token,
    overwrite=True,
)

# Verify credential file was created
with open(Path.home() / ".qiskit" / "qiskit-ibm.json", "r") as f_in:
    credential_file = f_in.read()
    print(credential_file)

In [ ]:
# Cell 2 - Display available backends

service = QiskitRuntimeService(channel="ibm_quantum")

for backend in service.backends(simulator=False, operational=True):
    config = backend.configuration()
    print(
        f"{config.backend_name:15}: Qubits = {config.n_qubits}: Gates = {config.basis_gates}"
    )


In [ ]:
# Cell 3 - Create a Bell State (|00> + |11>)

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

qc.draw(output="mpl")


In [ ]:
# Cell 4 - Prepare circuit for least busy IBM quantum computer

backend = service.least_busy(simulator=False, operational=True)
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

isa_circuit.draw("mpl", idle_wires=False)


In [ ]:
# Cell 5 - Run circuit on the IBM quantum computer

estimator = Estimator(backend)
estimator.options.resilience_level = 1
estimator.options.default_shots = 5000

observables_labels = ["IZ", "IX", "ZI", "XI", "ZZ", "XX"]
observables = [SparsePauliOp(label) for label in observables_labels]
mapped_observables = [
    observable.apply_layout(isa_circuit.layout) for observable in observables
]

job = estimator.run([(isa_circuit, mapped_observables)])

print(f">>> Job ID: {job.job_id()}")


In [ ]:
# Cell 5 (Alternate) - Run circuit on the IBM simulator

from qiskit_ibm_runtime.fake_provider import FakeAlmadenV2

backend = FakeAlmadenV2()
estimator = Estimator(backend)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

observables_labels = ["IZ", "IX", "ZI", "XI", "ZZ", "XX"]
observables = [SparsePauliOp(label) for label in observables_labels]
mapped_observables = [
    observable.apply_layout(isa_circuit.layout) for observable in observables
]

job = estimator.run([(isa_circuit, mapped_observables)])

print(f">>> Job ID: {job.job_id()}")


In [ ]:
# Cell 6 - Get job results and plot observables

job_result = job.result()
pub_result = job.result()[0]

values = pub_result.data.evs
errors = pub_result.data.stds

plt.plot(observables_labels, values, "-o")
plt.xlabel("Observables")
plt.ylabel("Values")
plt.show()
